In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
#from sklearn import cluster
from kneed import KneeLocator

In [51]:
#read the pkl file for movies
df = pd.read_pickle('./ABC/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./ABC/full_shows.pkl')

In [20]:
df.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
11,11,Family,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,[],"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,NaN,,Ballet Now
35,35,Family,Zog,NaN,Zog,"Based on the picture book by Julia Donaldson, ...","Based on the picture book by Julia Donaldson, ...","[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW124...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4feature, 4featured, abcme-shoulder...",2020-04-17 11:45:30,G,1621.0,NaN,,Zog
111,111,Family,Armadillo: Narrated By David Attenborough,NaN,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,In the heart of Brazil lives an animal so elus...,"[ABC TV, DOCUMENTARY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[Host Sir David Attenborough],"[abc1, uk, docs, factual, nature, wildlife, en...",2021-09-19 19:41:30,PG,3134.0,NaN,,Armadillo: Narrated By David Attenborough
137,137,Family,Stick Man,NaN,Stick Man,Stick Man lives in the family tree with his St...,Stick Man lives in the family tree with his St...,"[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW002...,"[Director Jeroen Jaspaert, Daniel Snaddon]","[abc4kids, family-viewing, 4xmas-features, abc...",2020-04-17 11:44:30,G,1626.0,NaN,,Stick Man
158,158,Family,The Snail And The Whale,NaN,The Snail And The Whale,"A beautiful tale of an unlikely friendship, a ...","A beautiful tale of an unlikely friendship, a ...","[ABC Kids, ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW177...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4featured, 4feature, 4family, 4uppe...",2020-04-17 11:46:30,G,1601.0,NaN,,The Snail And The Whale


In [53]:
df_2.head()

,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2
title,,,,,,,,,
199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[environment-day, refugee-day, education, prim..."
7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[pandemic, analysis, budget, flooding, vaccine..."
7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 136.0, 202.0, 115...",[ABC NEWS],"[parachuting, news, news24, duty, candidates, ..."
7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,"[1030.0, 1030.0]",[ABC NEWS],"[news, news24]"
72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing-abcme, abc3, family-viewing]"


In [22]:
#read json file activity for movies
activity = pd.read_json('./activities.json')

In [23]:
activity.head()
activity.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
1,85,85,85
2,119,119,119
3,101,101,101
4,104,104,104
5,91,91,91


## TF-IDF + kmeans clustering

In [24]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


13


In [35]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

10


In [25]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters

In [37]:
# Cluster the documents using KMeans
num_clusters_shows = 10
km = KMeans(n_clusters=num_clusters_shows)
km.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.head()

,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,cluster
title,,,,,,,,,,
199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[environment-day, refugee-day, education, prim...",8
7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[pandemic, analysis, budget, flooding, vaccine...",2
7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 136.0, 202.0, 115...",[ABC NEWS],"[parachuting, news, news24, duty, candidates, ...",5
7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,"[1030.0, 1030.0]",[ABC NEWS],"[news, news24]",7
72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing-abcme, abc3, family-viewing]",1


In [41]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'

In [42]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,cluster
237,237,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,Romeo & Juliet,11
404,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,5
424,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!,2
484,484,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End,5
595,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman,8


In [49]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index', 'level_0'], inplace=True)
df_2.head()

,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,cluster
0,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[environment-day, refugee-day, education, prim...",8
1,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[pandemic, analysis, budget, flooding, vaccine...",2
2,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 136.0, 202.0, 115...",[ABC NEWS],"[parachuting, news, news24, duty, candidates, ...",5
3,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,"[1030.0, 1030.0]",[ABC NEWS],"[news, news24]",7
4,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing-abcme, abc3, family-viewing]",1


In [47]:
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows.head()

,index,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,cluster
51,51,Australia Debates,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0, 2492.0]","[ABC TV Plus, PANEL & DISCUSSION]","[panel, a:older-sink-dink, interview, aussie, ...",8
52,52,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[Host Aaron Pedersen],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[abcme-shoulder, ns:be-informed, australia, fa...",9
53,53,Australia Remastered: Forces of Nature,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3230.0, 3260.0, 3195.0, 3240.0, 3230....","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[environment, abc1, natural-disaster, climate-...",4
54,54,Australia Remastered: Nature's Great Divide,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[Host Aaron Pedersen],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3240.0, 3200.0, 3240.0, 3240....","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, abc1, explore, factual, docuseri...",4
55,55,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3230....","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[abcme-shoulder, ns:be-informed, australia, fa...",4


In [28]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library.head()

Cluster 0: share aboriginal connection artists explores culture metal heart dance story driven indigenous australia
Cluster 1: love year finds new turns family young 2004 man life affair successful actor
Cluster 2: book based world donaldson julia axel scheffler friendship tale animated tour written children
Cluster 3: identity australian story fashion fielding extraordinary 2020 iconic women told pearl zog gadabout
Cluster 4: life set pina church drama story great stories ultimately earth lives characters celebrated
Cluster 5: attenborough david making natural sir scenes look series history takes museum monsters ancient
Cluster 6: school play special stick episode children art emma having join stay safe help
Cluster 7: disability people geography science live stage research million eruption 2010 new international stages
Cluster 8: music story australia queen record country narrated mission gospel sarah album intimate singer
Cluster 9: beach summer time stranded lost cadets kangaroo re

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
7,741.0,Documentary,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[Host Julia Zemiro, Brian Cox]","'abc1', 'science', 'docs', 'space', 'family-vi...",8.0,741,Dislike,2
9,446.0,Drama,Auto Focus,A successful TV star and former actor of Hogan...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Paul Schrader],"'abc1', 'abc2', 'drama', 'crime', 'biopic', 'c...",1.0,446,Dislike,3
11,504.0,Arts,Sleeping Beauty,The Australian Ballet presents Sleeping Beauty...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[],"'abc2', 'arts', 'entertainment', 'concert', 'b...",11.0,504,Like,4
17,122.0,Documentary,Cyber Dreaming,Graphic artist Stuart Campbell has developed h...,https://cdn.iview.abc.net.au/thumbs/i/ac/publi...,[],"'abc1', 'abcarts', 'arts', 'docs', 'aussie', '...",0.0,122,Play,2
31,20.0,Kids,Play School: Fighting The Coronavirus,Play School wants to say thank you to everyone...,https://cdn.iview.abc.net.au/thumbs/i/ck/CK202...,[],'abc4kids',6.0,20,Dislike,5


In [30]:
joined_activity = pd.merge(df, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
11,504.0,Arts,Sleeping Beauty,The Australian Ballet presents Sleeping Beauty...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[],"'abc2', 'arts', 'entertainment', 'concert', 'b...",11.0,504,Like,4
43,702.0,Family,David Attenborough's Natural History Museum Alive,This program takes us on a magical journey thr...,https://cdn.iview.abc.net.au/thumbs/i/zx/ZX970...,[Host Sir David Attenborough],"'abc2', 'docs', 'nature', 'uk', 'featured', 's...",5.0,702,Like,4
44,702.0,Drama,Primary Colours,An all star cast sees Governor Stanton as a pr...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW287...,[Director Mike Nichols],"'abc1', 'abc2', 'film', 'drama', 'politics', '...",7.0,702,Like,4
45,925.0,Documentary,David Attenborough's Tasmania,David Attenborough narrates the story of a vas...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW110...,[Host Sir David Attenborough],"'abc2', 'docs', 'nature', 'wildlife', 'environ...",5.0,925,Like,4
47,223.0,Drama,Triangle,Yacht passengers encounter mysterious weather ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,[Director Christopher Smith],"'abc1', 'abc2', 'drama', 'fantasy', 'sci-fi', ...",1.0,223,Like,5
49,43.0,Drama,Adaptation,A lovelorn screenwriter struggles to adapt 'Th...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Spike Jonze],"'abc1', 'abc2', 'drama', 'comedy-drama', 'surr...",4.0,43,Like,5
52,454.0,Drama,Paranoia,A boss blackmails one of his young employees t...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW288...,[Director Robert Luketic],"'abc1', 'film', 'drama', 'adaptation', 'thrill...",12.0,454,Like,2
59,998.0,Drama,Mr Deeds Goes To Town,A small-town poet inherits a vast fortune and ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,"[Cast Gary Cooper, Jean Arthur, Douglas Dumbri...","'abc1', 'abc2', 'drama', 'comedy-drama', 'film...",1.0,998,Like,5
71,512.0,Arts,La Sylphide,"The Australian Ballet presents La Sylphide, a ...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[],"'abc2', 'arts', 'abcarts', 'ballet', 'performa...",11.0,512,Like,4
85,476.0,Arts,Intimate Encounters: 20 Years On,"In 1988, an exhibition caused shockwaves with ...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK209...,[Director Dieter Knierim],"'abc1', 'docs', 'short', 'arts', 'abcarts', 's...",7.0,476,Like,1


In [31]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes

,user_id,cluster
0,1,5.0
1,1,7.0
2,1,8.0
3,2,5.0
4,2,8.0
5,2,12.0
6,3,3.0
7,3,1.0
8,3,6.0
9,4,5.0


In [32]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id


In [33]:
#filter expat_data_likes dataframe by another dataframe with two columns: user_id and cluster
#this is only activity 'Like' for top-3 clusters for each user
expat_data_top = expat_data_likes[expat_data_likes[['user_id', 'cluster']].apply(tuple, axis=1).isin(top3.index)]
expat_data_top

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,cluster
237,237,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,Romeo & Juliet,11
404,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,5
424,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!,2
484,484,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End,5
595,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,871,Arts,The Art of Collecting,NaN,The Art of Collecting,These short intimate stories by artist Kate Bl...,These short intimate stories by artist Kate Bl...,"[ABC TV Plus, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/ac/AC194...,"[Cast David Capra, Sarah Goffman, Karla Dicken...","[abc2, australia, arts, culture, inspirational...",2021-06-16 22:31:40,G,1933.0,NaN,,The Art of Collecting,0
897,897,Arts,Whiteley,NaN,Whiteley,A visual journey into the private life and cre...,A visual journey into the private life and cre...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW138...,[Director James Bogle],"[abc1, abc2, abcarts, arts, docs, history, bio...",2021-06-01 07:00:00,M,5395.0,NaN,,Whiteley,3
209,209,Education,"Tell Your Story, Change Your World",NaN,"Tell Your Story, Change Your World","Only you can tell your own story, but getting ...","Only you can tell your own story, but getting ...","[ABC ME, EDUCATION]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK189...,[],"[abc3, education, secondary-english]",2018-10-19 10:42:31,PG,875.0,NaN,,"Tell Your Story, Change Your World",4
831,831,Education,Deadly Family Portraits: Crombie Crew,NaN,Deadly Family Portraits: Crombie Crew,Two Aboriginal artists Lillian and Elaine Crom...,Two Aboriginal artists Lillian and Elaine Crom...,"[ABC TV, ABC ARTS, INDIGENOUS, DOCUMENTARY, AR...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC182...,[],"[abc1, abcarts, indigenous, docs, arts, cultur...",2019-08-09 07:00:00,G,603.0,NaN,,Deadly Family Portraits: Crombie Crew,0


In [34]:
#check what from expat_library is not in expat_data_top by content_id
#that's ti check that we don't recommend movies that user already watched
suggestion = expat_library[~expat_library['id'].isin(expat_data_top['id'])]
suggestion

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,5.0,484.0,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End
1,1,5.0,406.0,Documentary,Census: Why We Count (2021),NaN,Census: Why We Count (2021),The 2021 Australian Census is on Tuesday 10 Au...,The 2021 Australian Census is on Tuesday 10 Au...,"[ABC TV, ABC NEWS, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Casey Briggs],"[abc1, news, docs, current-affairs, culture, a...",2021-08-08 18:29:54,PG,1758.0,NaN,,Census: Why We Count ()
2,1,7.0,475.0,Arts,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DRAMA, COME...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Bridie McKim],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,Re-Frame
3,1,7.0,590.0,Drama,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DRAMA, COME...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Bridie McKim],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,Re-Frame
4,1,8.0,741.0,Documentary,Stargazing: Moon and Beyond,NaN,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,Stargazing celebrates the 50th anniversary of ...,"[ABC TV, SCIENCE, DOCUMENTARY, FAMILY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[Host Julia Zemiro, Brian Cox]","[abc1, science, docs, space, family-viewing, e...",2019-07-16 21:01:50,PG,3599.0,NaN,,Stargazing: Moon and Beyond
5,1,8.0,595.0,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
6,2,5.0,484.0,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End
7,2,5.0,406.0,Documentary,Census: Why We Count (2021),NaN,Census: Why We Count (2021),The 2021 Australian Census is on Tuesday 10 Au...,The 2021 Australian Census is on Tuesday 10 Au...,"[ABC TV, ABC NEWS, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Casey Briggs],"[abc1, news, docs, current-affairs, culture, a...",2021-08-08 18:29:54,PG,1758.0,NaN,,Census: Why We Count ()
8,2,8.0,741.0,Documentary,Stargazing: Moon and Beyond,NaN,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,Stargazing celebrates the 50th anniversary of ...,"[ABC TV, SCIENCE, DOCUMENTARY, FAMILY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[Host Julia Zemiro, Brian Cox]","[abc1, science, docs, space, family-viewing, e...",2019-07-16 21:01:50,PG,3599.0,NaN,,Stargazing: Moon and Beyond
9,2,8.0,595.0,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factua